In [1]:
import numpy as np
import pandas as pd

# Load dataset
data = pd.read_csv('asiacup.csv')

# Preprocessing: Replace categorical values with numerical ones
data['Result'] = data['Result'].apply(lambda x: 1 if x == 'Win' else 0)

# Selecting features and target
X = data[['Run Scored', 'Wicket Lost', 'Fours', 'Sixes', 'Extras']].values
y = data['Result'].values.reshape(-1, 1)

# Handling missing values
X = np.nan_to_num(X)
y = np.nan_to_num(y)

# Normalize the input data for better performance
X = (X - np.min(X, axis=0)) / (np.max(X, axis=0) - np.min(X, axis=0))

# Kohonen Layer (Unsupervised Learning)
input_neurons = X.shape[1]
kohonen_neurons = 10  # You can adjust this value
learning_rate = 0.01
epochs = 10000

# Initialize weights between input and Kohonen layer
weights_input_kohonen = np.random.uniform(-0.5, 0.5, (input_neurons, kohonen_neurons))

def euclidean_distance(x, weights):
    return np.sqrt(np.sum((x - weights) ** 2, axis=1))

# Training Kohonen Layer (Unsupervised)
for epoch in range(epochs):
    for i in range(X.shape[0]):
        input_vector = X[i]
        
        # Compute distance from input to all Kohonen neurons
        distances = euclidean_distance(input_vector, weights_input_kohonen.T)
        
        # Find the winning neuron (nearest Kohonen neuron)
        winning_neuron = np.argmin(distances)
        
        # Update weights of the winning neuron
        weights_input_kohonen[:, winning_neuron] += learning_rate * (input_vector - weights_input_kohonen[:, winning_neuron])

# Grossberg Layer (Supervised Learning)
grossberg_neurons = 1  # Output is binary (Win/Lose)

# Initialize weights between Kohonen and Grossberg layers
weights_kohonen_grossberg = np.random.uniform(-0.5, 0.5, (kohonen_neurons, grossberg_neurons))

# Sigmoid function and its derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# Supervised learning in Grossberg Layer
for epoch in range(epochs):
    for i in range(X.shape[0]):
        input_vector = X[i]
        
        # Kohonen Layer output (Winner-takes-all)
        distances = euclidean_distance(input_vector, weights_input_kohonen.T)
        kohonen_output = np.zeros(kohonen_neurons)
        kohonen_output[np.argmin(distances)] = 1  # Only the winning neuron is activated
        
        # Forward pass through Grossberg Layer
        grossberg_input = np.dot(kohonen_output, weights_kohonen_grossberg)
        grossberg_output = sigmoid(grossberg_input)
        
        # Compute error
        error = y[i] - grossberg_output
        
        # Backpropagation (update weights between Kohonen and Grossberg layers)
        d_grossberg_output = error * sigmoid_derivative(grossberg_output)
        weights_kohonen_grossberg += np.outer(kohonen_output, d_grossberg_output) * learning_rate
    
    if epoch % 1000 == 0:
        mean_squared_error = np.mean(np.square(y - sigmoid(np.dot((X @ weights_input_kohonen), weights_kohonen_grossberg))))
        print(f"Epoch {epoch} Error: {mean_squared_error}")

# After training, calculate accuracy
final_output = sigmoid(np.dot((X @ weights_input_kohonen), weights_kohonen_grossberg))
predicted_output = np.round(final_output)

accuracy = np.mean(predicted_output == y) * 100
print(f"Accuracy: {accuracy}%")


Epoch 0 Error: 0.3097568861534379
Epoch 1000 Error: 0.19922677514808104
Epoch 2000 Error: 0.19923062671322206
Epoch 3000 Error: 0.19922881043218277
Epoch 4000 Error: 0.19922869951297884
Epoch 5000 Error: 0.19922869464492513
Epoch 6000 Error: 0.19922869445664254
Epoch 7000 Error: 0.19922869444982721
Epoch 8000 Error: 0.19922869444957744
Epoch 9000 Error: 0.19922869444956645
Accuracy: 70.07874015748031%
